### Importing Data

### Importing libraries

In [ ]:
import pathlib
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau
from glob import glob
import random

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


This assignment uses a dataset of about 2357 images of skin cancer types. The dataset contains 9 sub-directories in each train and test subdirectories. The 9 sub-directories contains the images of 9 skin cancer types respectively.

In [ ]:
# Defining the path for train and test images
data_dir_train = pathlib.Path("/content/gdrive/MyDrive/AI_ML_DS/Google_Colab_Workspace/Skin cancer ISIC The International Skin Imaging Collaboration/Train")
data_dir_test = pathlib.Path('/content/gdrive/MyDrive/AI_ML_DS/Google_Colab_Workspace/Skin cancer ISIC The International Skin Imaging Collaboration/Test')

In [ ]:
image_count_train = len(list(data_dir_train.glob('*/*.jpg')))
print(image_count_train)

In [ ]:
image_count_test = len(list(data_dir_test.glob('*/*.jpg')))
print(image_count_test)

### Load using keras.preprocessing

### Define some parameters for the loader:

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

Pushing 80% of the images for training, and rest for validation.

In [ ]:
## Write your train dataset here
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir_train, labels='inferred', label_mode='int',
    class_names=None, color_mode='rgb', batch_size=32, image_size=(img_height,
    img_width), shuffle=True, seed=123, validation_split=0.2, subset='training',
    interpolation='bilinear', follow_links=False
)

In [ ]:
## Write your validation dataset here
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir_train, labels='inferred', label_mode='int',
    class_names=None, color_mode='rgb', batch_size=32, image_size=(img_height,
    img_width), shuffle=True, seed=123, validation_split=0.2, subset='validation',
    interpolation='bilinear', follow_links=False
)

In [ ]:
# List out all the classes of skin cancer and store them in a list.
# You can find the class names in the class_names attribute on these datasets.
# These correspond to the directory names in alphabetical order.
class_names = train_ds.class_names
print(class_names)

### Visualize the data

In [ ]:
### your code goes here, you can use training or validation data to visualize

def generate_image_map(image_ds,btch_num=1):
  # Dictionary to store one image per label (0 to 9)
  label_image_map = {}
  # Iterate through the dataset without limiting the number of batches
  for images, labels in image_ds.skip(btch_num-1).take(1):
      # Loop through each image and its corresponding label
      for image, label in zip(images.numpy(), labels.numpy()):
          # If we haven't already stored an image for this label
          if label not in label_image_map:
              label_image_map[label] = image  # Store the image with the unique label
          # Stop once we have one image for each label (0 to 9)
          if len(label_image_map) == 10:
              break  # Exit the inner loop if all 10 labels are found
      if len(label_image_map) == 10:
          break  # Exit outer loop once all labels are covered
  # Sort the dictionary by keys
  sorted_image_map=dict(sorted(label_image_map.items()))
  # Display the keys of the dictionary to check which labels have been collected
  print("Labels found:", sorted_image_map.keys())
  return sorted_image_map


def plt_sample_img(class_names, label_image_map):
  plt.figure(figsize=(20, 10))
  for idx, (label, image) in enumerate(label_image_map.items()):
      plt.subplot(2, 5, idx + 1)  # Create a 2x5 grid for the images
      plt.imshow(image.astype("uint8"))  # Display the image
      plt.title(f"Class: {class_names[label]}")  # Display the class name
      plt.axis('off')  # Turn off the axis
  # Show the plot with all images
  plt.tight_layout()
  plt.show()

In [ ]:
# Genarate random batch number between 0 and 10
btch_num=random.randint(1,11)
btch_num

In [ ]:
# Visualize Train Data
train_image_map=generate_image_map(train_ds,btch_num)
plt_sample_img(class_names,train_image_map)

In [ ]:
# Visualize Validation Data
val_image_map=generate_image_map(val_ds,btch_num)
plt_sample_img(class_names,val_image_map)

The `image_batch` is a tensor of the shape `(32, 180, 180, 3)`. This is a batch of 32 images of shape `180x180x3` (the last dimension refers to color channels RGB). The `label_batch` is a tensor of the shape `(32,)`, these are corresponding labels to the 32 images.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Model 1 - Base Model

### Create the model

In [ ]:
### Your code goes here
base_model=Sequential(name="Melanoma_Detection_Model")
# Adding Rescaling layer
base_model.add(layers.Rescaling(1.0/255.0 , offset=0.0 , input_shape=(img_height,img_width,3), name="Rescaling_Layer"))
# First Part
base_model.add(layers.Conv2D(filters=16,kernel_size=(3,3),padding="same",activation="relu",  name="Conv2D_Layer_1"))  # adding first Convolution layer
base_model.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_1")) # adding first Max Pooling layer
# Second Part
base_model.add(layers.Conv2D(filters=32,kernel_size=(3,3),padding="same",activation="relu", name="Conv2D_Layer_2")) # adding second Convolution layer
base_model.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_2")) # adding second Max Pooling layer
# Third Part
base_model.add(layers.Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu", name="Conv2D_Layer_3")) # adding second Convolution layer
base_model.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_3")) # adding second Max Pooling layer
# Flattening the output
base_model.add(layers.Flatten(name="Flatten_Layer"))
# Fully Connected Layer
base_model.add(layers.Dense(units=128,activation="relu",name="FC_Layer_1") )# adding first fully connected layer
# Output Layer
base_model.add(layers.Dense(units=len(class_names),activation="softmax",name="Output_Layer")) # adding output layer

### Compile the model

In [ ]:
### Todo, choose an appropirate optimiser and loss function
base_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# View the summary of all layers
base_model.summary()

### Train the model

In [ ]:
epochs = 20
base_model_history = base_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

### Visualizing training results

In [ ]:
acc = base_model_history.history['accuracy']
val_acc = base_model_history.history['val_accuracy']

loss = base_model_history.history['loss']
val_loss = base_model_history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

#### Todo: Write your findings after the model fit, see if there is an evidence of model overfit or underfit

### Write your findings here
- Training accuracy steadily improve to 88-90% by the 20th epoch, indicating that - the model is learning and fitting the training data well.
- The validation accuracy, on the other hand, peaks at around 55% and - then stagnates or decreases by the 20th epoch. This shows that the model - is not generalizing well to unseen data and starts to overfit after a - certain point.
- While the training loss continues to drop, the validation loss increases significantly over time by the 20th epoch.

The model is overfitting, as indicated by the widening gap between the training and validation performance

## Model 2 - Base Model with Data Augmentation

### Data augmentation

In [ ]:
data_augmentation = Sequential([
  layers.RandomFlip("horizontal_and_vertical", seed=123 ),
  layers.RandomRotation(0.2, seed=123),
  layers.RandomZoom(0.2, seed=123),
] , name="Data_Augmentation_Layer")

In [ ]:
random_num=random.randint(1,11)
plt.figure(figsize=(10, 8))
for images, _ in train_ds.skip(random_num-1).take(random_num):
  for i in range(9):
    augmented_images = data_augmentation(images) # data augmenatation
    ax = plt.subplot(3, 3, i + 1) # Create a 3x3 grid for the images
    plt.imshow(augmented_images[random_num].numpy().astype("uint8")) # Display the image
    plt.axis("off") # Turn off the axis
plt.tight_layout()
plt.show()

### Create the model, compile and train the model


In [ ]:
aug_layer_model=Sequential(name="Melanoma_Detection_Model")
# Adding Rescaling layer
aug_layer_model.add(layers.Rescaling(1.0/255.0 , offset=0.0 , input_shape=(img_height,img_width,3), name="Rescaling_Layer"))
# Adding Data Augmentation
aug_layer_model.add( data_augmentation )
# First Part
aug_layer_model.add(layers.Conv2D(filters=16,kernel_size=(3,3),padding="same",activation="relu",  name="Conv2D_Layer_1"))  # adding first Convolution layer
aug_layer_model.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_1")) # adding first Max Pooling layer
# Second Part
aug_layer_model.add(layers.Conv2D(filters=32,kernel_size=(3,3),padding="same",activation="relu", name="Conv2D_Layer_2")) # adding second Convolution layer
aug_layer_model.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_2")) # adding second Max Pooling layer
# Third Part
aug_layer_model.add(layers.Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu", name="Conv2D_Layer_3")) # adding second Convolution layer
aug_layer_model.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_3")) # adding second Max Pooling layer
aug_layer_model.add(layers.Dropout(0.30 ,name="Dropout_1") )# adding dropouts
# Flattening the output
aug_layer_model.add(layers.Flatten(name="Flatten_Layer"))
# Fully Connected Layer
aug_layer_model.add(layers.Dense(units=128,activation="relu",name="FC_Layer_1") )# adding first fully connected layer
aug_layer_model.add(layers.Dropout(0.30,name="Dropout_2")) # adding dropouts
# Output Layer
aug_layer_model.add(layers.Dense(units=len(class_names),activation="softmax",name="Output_Layer")) # adding output layer

### Compiling the model

In [ ]:
aug_layer_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
aug_layer_model.summary()

### Training the model

In [ ]:
epochs = 20
aug_layer_history = aug_layer_model.fit(train_ds ,
                    batch_size=batch_size ,
                    epochs=epochs,
                    validation_data=val_ds,
                    verbose=1)

### Visualizing the results

In [ ]:
acc = aug_layer_history.history['accuracy']
val_acc = aug_layer_history.history['val_accuracy']

loss = aug_layer_history.history['loss']
val_loss = aug_layer_history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Summary:
- Training accuracy and validation accuracy are almost the same but are both quite low, indicating that the model is likely underfitting.
- Training loss is very low, while the validation loss fluctuates significantly.
- These results suggest that the current model is not performing well and requires improvement.
- To add on, addressing the issue of class imbalance could further - improve the model's performance.

## Model 3 - Base Model with Data Augmentation after fixing Data imbalacing

### **Todo:** Find the distribution of classes in the training dataset.

In [ ]:
list_images = []
for i in class_names:
    list_images.append(len(list(data_dir_train.glob(i+'/*.jpg'))))

data = {'Class Names': class_names, 'Image Count': list_images}
original_df = pd.DataFrame(data)

In [ ]:
original_df.head(9)

#### - Which class has the least number of samples?
Answer - seborrheic keratosis - 77
#### - Which classes dominate the data in terms proportionate number of samples?
Answer - pigmented benign keratosis - 462


#### Rectify the class imbalance

In [ ]:
!pip install Augmentor

To use `Augmentor`, the following general procedure is followed:

1. Instantiate a `Pipeline` object pointing to a directory containing your initial image data set.<br>
2. Define a number of operations to perform on this data set using your `Pipeline` object.<br>
3. Execute these operations by calling the `Pipeline’s` `sample()` method.


In [ ]:
path_to_training_dataset="/content/gdrive/MyDrive/AI_ML_DS/Google_Colab_Workspace/Skin cancer ISIC The International Skin Imaging Collaboration/Train/"
import Augmentor
for i in class_names:
    class_path = path_to_training_dataset + i + '/output'
    # Check if 'output' folder exists and has the required number of images
    if os.path.exists(class_path) and len(os.listdir(class_path)) >= target_sample_size:
        print(f"Augmentation already done for class '{i}', skipping augmentation.")
    else:
        print(f"Augmenting class '{i}' as required samples are not present.")
    # Create an Augmentor pipeline for the class if augmentation is needed
    p = Augmentor.Pipeline(path_to_training_dataset + i)
    p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
    p.sample(500) ## We are adding 500 samples per class to make sure that none of the classes are sparse.

Augmentor has stored the augmented images in the output sub-directory of each of the sub-directories of skin cancer types.. Lets take a look at total count of augmented images.

In [ ]:
image_count_train = len(list(data_dir_train.glob('*/output/*.jpg')))
print(image_count_train)

### Lets see the distribution of augmented data after adding new images to the original training data.

In [ ]:
path_list_new = [x for x in glob(os.path.join(data_dir_train, '*','output', '*.jpg'))]
path_list_new

In [ ]:
lesion_list_new = [os.path.basename(os.path.dirname(os.path.dirname(y))) for y in glob(os.path.join(data_dir_train, '*','output', '*.jpg'))]
lesion_list_new

In [ ]:
dataframe_dict_new = dict(zip(path_list_new, lesion_list_new))

In [ ]:
df2 = pd.DataFrame(list(dataframe_dict_new.items()),columns = ['Path','Label'])
new_df = pd.concat([original_df,df2],ignore_index=True)

In [ ]:
new_df['Label'].value_counts()

So, now we have added 500 images to all the classes to maintain some class balance. We can add more images as we want to improve training process.

### **Todo**: Train the model on the data created using Augmentor

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

#### **Todo:** Create a training dataset

In [ ]:
data_dir_train="/content/gdrive/MyDrive/AI_ML_DS/Google_Colab_Workspace/Skin cancer ISIC The International Skin Imaging Collaboration/Train/"
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  seed=123,
  validation_split = 0.2,
  subset = 'training',
  image_size=(img_height, img_width),
  batch_size=batch_size)

#### **Todo:** Create a validation dataset

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  seed=123,
  validation_split = 0.2,
  subset ='validation',
  image_size=(img_height, img_width),
  batch_size=batch_size)

### **Todo:** Create your model (make sure to include normalization)

#### Create your model

In [ ]:
class_blncd_mdl = Sequential(name="Melanoma_Detection_Model")

# Adding Rescaling layer
class_blncd_mdl.add(layers.Rescaling(1.0/255.0, offset=0.0, input_shape=(img_height, img_width, 3), name="Rescaling_Layer"))

# Adding Data Augmentation
class_blncd_mdl.add(data_augmentation)

# First Part
class_blncd_mdl.add(layers.Conv2D(filters=16, kernel_size=(3,3),padding="same", name="Conv2D_Layer_1"))  # adding first Convolution layer
class_blncd_mdl.add(layers.BatchNormalization(name="BatchNorm_Layer_1"))  # adding normalization
class_blncd_mdl.add(layers.Activation('relu', name="ReLU_Activation_1"))
class_blncd_mdl.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_1"))  # adding first Max Pooling layer

# Second Part
class_blncd_mdl.add(layers.Conv2D(filters=32, kernel_size=(3,3),padding="same", name="Conv2D_Layer_2"))  # adding second Convolution layer
class_blncd_mdl.add(layers.BatchNormalization(name="BatchNorm_Layer_2"))  # adding normalization
class_blncd_mdl.add(layers.Activation('relu', name="ReLU_Activation_2"))
class_blncd_mdl.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_2"))  # adding second Max Pooling layer

# Third Part
class_blncd_mdl.add(layers.Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu", name="Conv2D_Layer_3")) # adding second Convolution layer
class_blncd_mdl.add(layers.BatchNormalization(name="BatchNorm_Layer_3"))  # adding normalization
class_blncd_mdl.add(layers.Activation('relu', name="ReLU_Activation_3"))
class_blncd_mdl.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_3")) # adding second Max Pooling layer
class_blncd_mdl.add(layers.Dropout(0.30 ,name="Dropout_1") )# adding dropouts

# Flattening the output
class_blncd_mdl.add(layers.Flatten(name="Flatten_Layer"))

# Fully Connected Layer
class_blncd_mdl.add(layers.Dense(units=128, name="FC_Layer_1"))
class_blncd_mdl.add(layers.BatchNormalization(name="BatchNorm_Layer_4"))  # adding normalization
class_blncd_mdl.add(layers.Activation('relu', name="ReLU_Activation_4"))
class_blncd_mdl.add(layers.Dropout(0.30, name="Dropout_2"))  # adding dropout

# Output Layer
class_blncd_mdl.add(layers.Dense(units=len(class_names), activation="softmax", name="Output_Layer"))  # adding output layer

#### **Todo:** Compile your model (Choose optimizer and loss function appropriately)

In [ ]:
class_blncd_mdl.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# View the summary of all layers
class_blncd_mdl.summary()

#### **Todo:**  Train your model

In [ ]:
epochs = 30
## Your code goes here, use 30 epochs.
class_balanced_history = class_blncd_mdl.fit(train_ds ,
                    batch_size=batch_size ,
                    epochs=epochs,
                    validation_data=val_ds,
                    verbose=1)

#### **Todo:**  Visualize the model results

In [ ]:
acc = class_balanced_history.history['accuracy']
val_acc = class_balanced_history.history['val_accuracy']

loss = class_balanced_history.history['loss']
val_loss = class_balanced_history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Model 4 - Include Callbacks and adjust Dropouts ( gradually increase dropout percentages )

In [ ]:
## your code goes here
lr_control_model = Sequential(name="Melanoma_Detection_Model")

# Adding Rescaling layer
lr_control_model.add(layers.Rescaling(1.0/255.0, offset=0.0, input_shape=(img_height, img_width, 3), name="Rescaling_Layer"))

# Adding Data Augmentation
lr_control_model.add(data_augmentation)

# First Part
lr_control_model.add(layers.Conv2D(filters=16, kernel_size=(3,3), padding="same",  name="Conv2D_Layer_1"))  # adding first Convolution layer
lr_control_model.add(layers.BatchNormalization(name="BatchNorm_Layer_1"))  # adding normalization
lr_control_model.add(layers.Activation('relu', name="ReLU_Activation_1"))
lr_control_model.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_1"))  # adding first Max Pooling layer

# Second Part
lr_control_model.add(layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", name="Conv2D_Layer_2"))  # adding second Convolution layer
lr_control_model.add(layers.BatchNormalization(name="BatchNorm_Layer_2"))  # adding normalization
lr_control_model.add(layers.Activation('relu', name="ReLU_Activation_2"))
lr_control_model.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_2"))  # adding second Max Pooling layer

# Third Part
lr_control_model.add(layers.Conv2D(filters=64,kernel_size=(3,3), padding="same",activation="relu", name="Conv2D_Layer_3")) # adding second Convolution layer
lr_control_model.add(layers.BatchNormalization(name="BatchNorm_Layer_3"))  # adding normalization
lr_control_model.add(layers.Activation('relu', name="ReLU_Activation_3"))
lr_control_model.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_3")) # adding second Max Pooling layer
lr_control_model.add(layers.Dropout(0.30 ,name="Dropout_1") )# adding dropouts

# Flattening the output
lr_control_model.add(layers.Flatten(name="Flatten_Layer"))

# Fully Connected Layer
lr_control_model.add(layers.Dense(units=128, name="FC_Layer_1"))
lr_control_model.add(layers.BatchNormalization(name="BatchNorm_Layer_4"))  # adding normalization
lr_control_model.add(layers.Activation('relu', name="ReLU_Activation_4"))
lr_control_model.add(layers.Dropout(0.40, name="Dropout_2"))  # adding dropout

# Output Layer
lr_control_model.add(layers.Dense(units=len(class_names), activation="softmax", name="Output_Layer"))  # adding output layer

# Compile the model
lr_control_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Set callbacks
learn_control = ReduceLROnPlateau(monitor='val_accuracy',
                                  patience=3,
                                  verbose=1,
                                  factor=0.1,
                                  min_lr=1e-7)

# View the summary of all layers
lr_control_model.summary()

Train the model

In [ ]:
epochs = 30
lr_control_history = lr_control_model.fit(train_ds ,
                    batch_size=batch_size ,
                    epochs=epochs,
                    validation_data=val_ds,
                    callbacks=[learn_control],
                    verbose=1)

Visualize the model results

In [ ]:
acc = lr_control_history.history['accuracy']
val_acc = lr_control_history.history['val_accuracy']

loss = lr_control_history.history['loss']
val_loss = lr_control_history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

#### Analyze your results here. Did you get rid of underfitting/overfitting? Did class rebalance help?

In Model 1, we observed significant overfitting, with high training accuracy (0.84) but much lower validation accuracy (0.55). Adding data augmentation in Model 2 helped reduce the gap between training and validation accuracy, but underfitting became a concern. Model 3, with class rebalancing, further reduced overfitting but slightly hindered validation performance.

Model 4, which introduced controlled learning rate, showed improved generalization, as indicated by a more balanced train and validation accuracy (0.63 vs. 0.61). Overall, class rebalancing and learning rate control contributed positively to the model’s stability.

## Model 5 - Let's try with little increased Learning rate and also include Callbacks

Create the Model

In [ ]:
increased_lr_ctrl_mdl = Sequential(name="Melanoma_Detection_Model")

# Adding Rescaling layer
increased_lr_ctrl_mdl.add(layers.Rescaling(1.0/255.0, offset=0.0, input_shape=(img_height, img_width, 3), name="Rescaling_Layer"))

# Adding Data Augmentation
increased_lr_ctrl_mdl.add(data_augmentation)

# First Part
increased_lr_ctrl_mdl.add(layers.Conv2D(filters=16, kernel_size=(3,3), padding="same",  name="Conv2D_Layer_1"))  # adding first Convolution layer
increased_lr_ctrl_mdl.add(layers.BatchNormalization(name="BatchNorm_Layer_1"))  # adding normalization
increased_lr_ctrl_mdl.add(layers.Activation('relu', name="ReLU_Activation_1"))
increased_lr_ctrl_mdl.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_1"))  # adding first Max Pooling layer

# Second Part
increased_lr_ctrl_mdl.add(layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", name="Conv2D_Layer_2"))  # adding second Convolution layer
increased_lr_ctrl_mdl.add(layers.BatchNormalization(name="BatchNorm_Layer_2"))  # adding normalization
increased_lr_ctrl_mdl.add(layers.Activation('relu', name="ReLU_Activation_2"))
increased_lr_ctrl_mdl.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_2"))  # adding second Max Pooling layer

# Third Part
increased_lr_ctrl_mdl.add(layers.Conv2D(filters=64,kernel_size=(3,3), padding="same",activation="relu", name="Conv2D_Layer_3")) # adding second Convolution layer
increased_lr_ctrl_mdl.add(layers.BatchNormalization(name="BatchNorm_Layer_3"))  # adding normalization
increased_lr_ctrl_mdl.add(layers.Activation('relu', name="ReLU_Activation_3"))
increased_lr_ctrl_mdl.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_3")) # adding second Max Pooling layer
increased_lr_ctrl_mdl.add(layers.Dropout(0.30 ,name="Dropout_1") )# adding dropouts

# Flattening the output
increased_lr_ctrl_mdl.add(layers.Flatten(name="Flatten_Layer"))

# Fully Connected Layer
increased_lr_ctrl_mdl.add(layers.Dense(units=128, name="FC_Layer_1"))
increased_lr_ctrl_mdl.add(layers.BatchNormalization(name="BatchNorm_Layer_4"))  # adding normalization
increased_lr_ctrl_mdl.add(layers.Activation('relu', name="ReLU_Activation_4"))
increased_lr_ctrl_mdl.add(layers.Dropout(0.40, name="Dropout_2"))  # adding dropout

# Output Layer
increased_lr_ctrl_mdl.add(layers.Dense(units=len(class_names), activation="softmax", name="Output_Layer"))  # adding output layer

# Compile the model
increased_lr_ctrl_mdl.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), # Reduce learning rate
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# Set callbacks
learn_control = ReduceLROnPlateau(monitor='val_accuracy',
                                  patience=3,
                                  verbose=1,
                                  factor=0.1,
                                  min_lr=1e-7)

# View the summary of all layers
increased_lr_ctrl_mdl.summary()

Train the model

In [ ]:
epochs = 30
increased_lr_control_history = increased_lr_ctrl_mdl.fit(train_ds ,
                    batch_size=batch_size ,
                    epochs=epochs,
                    validation_data=val_ds,
                    callbacks=[learn_control],
                    verbose=1)

Visualize the model results

In [ ]:
acc = increased_lr_control_history.history['accuracy']
val_acc = increased_lr_control_history.history['val_accuracy']

loss = increased_lr_control_history.history['loss']
val_loss = increased_lr_control_history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Model 6 - Default Learning Rate and increased Epoch Value

In [ ]:
more_epoch_model = Sequential(name="Melanoma_Detection_Model")

# Adding Rescaling layer
more_epoch_model.add(layers.Rescaling(1.0/255.0, offset=0.0, input_shape=(img_height, img_width, 3), name="Rescaling_Layer"))

# Adding Data Augmentation
more_epoch_model.add(data_augmentation)

# First Part
more_epoch_model.add(layers.Conv2D(filters=16, kernel_size=(3,3), padding="same",  name="Conv2D_Layer_1"))  # adding first Convolution layer
more_epoch_model.add(layers.BatchNormalization(name="BatchNorm_Layer_1"))  # adding normalization
more_epoch_model.add(layers.Activation('relu', name="ReLU_Activation_1"))
more_epoch_model.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_1"))  # adding first Max Pooling layer

# Second Part
more_epoch_model.add(layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", name="Conv2D_Layer_2"))  # adding second Convolution layer
more_epoch_model.add(layers.BatchNormalization(name="BatchNorm_Layer_2"))  # adding normalization
more_epoch_model.add(layers.Activation('relu', name="ReLU_Activation_2"))
more_epoch_model.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_2"))  # adding second Max Pooling layer

# Third Part
more_epoch_model.add(layers.Conv2D(filters=64,kernel_size=(3,3), padding="same",activation="relu", name="Conv2D_Layer_3")) # adding second Convolution layer
more_epoch_model.add(layers.BatchNormalization(name="BatchNorm_Layer_3"))  # adding normalization
more_epoch_model.add(layers.Activation('relu', name="ReLU_Activation_3"))
more_epoch_model.add(layers.MaxPooling2D(pool_size=(2,2), name="MaxPooling2D_Layer_3")) # adding second Max Pooling layer
more_epoch_model.add(layers.Dropout(0.30 ,name="Dropout_1") )# adding dropouts

# Flattening the output
more_epoch_model.add(layers.Flatten(name="Flatten_Layer"))

# Fully Connected Layer
more_epoch_model.add(layers.Dense(units=128, name="FC_Layer_1"))
more_epoch_model.add(layers.BatchNormalization(name="BatchNorm_Layer_4"))  # adding normalization
more_epoch_model.add(layers.Activation('relu', name="ReLU_Activation_4"))
more_epoch_model.add(layers.Dropout(0.40, name="Dropout_2"))  # adding dropout

# Output Layer
more_epoch_model.add(layers.Dense(units=len(class_names), activation="softmax", name="Output_Layer"))  # adding output layer

# Compile the model
more_epoch_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Set callbacks
learn_control = ReduceLROnPlateau(monitor='val_accuracy',
                                  patience=3,
                                  verbose=1,
                                  factor=0.1,
                                  min_lr=1e-7)

# View the summary of all layers
more_epoch_model.summary()

Train the model

In [ ]:
epochs = 50
more_epoch_history = more_epoch_model.fit(train_ds ,
                    batch_size=batch_size ,
                    epochs=epochs,
                    validation_data=val_ds,
                    callbacks=[learn_control],
                    verbose=1)

Visualize the model results

In [ ]:
acc = more_epoch_history.history['accuracy']
val_acc = more_epoch_history.history['val_accuracy']

loss = more_epoch_history.history['loss']
val_loss = more_epoch_history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Model 7 - Six Convolution layers with Batch Normalization and dropouts
Max Pooling layer after every two Convolution Layers

Create the Model

In [ ]:
# Create the Model
six_layers_model = Sequential(name="Melanoma_Detection_Model_More_Layers")

# Adding Rescaling layer
six_layers_model.add(layers.Rescaling(1.0/255.0, offset=0.0, input_shape=(img_height, img_width, 3), name="Rescaling_Layer"))

# Adding Data Augmentation
six_layers_model.add(data_augmentation)

# First Part
six_layers_model.add(layers.Conv2D(filters=16, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_16F_1"))  # adding Convolution layer
six_layers_model.add(layers.BatchNormalization())  # adding normalization
six_layers_model.add(layers.Activation('relu'))
six_layers_model.add(layers.Conv2D(filters=16, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_16F_2"))  # adding Convolution layer
six_layers_model.add(layers.BatchNormalization())  # adding normalization
six_layers_model.add(layers.Activation('relu'))
six_layers_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Second Part
six_layers_model.add(layers.Conv2D(filters=32, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_32F_1"))  # adding Convolution layer
six_layers_model.add(layers.BatchNormalization())  # adding normalization
six_layers_model.add(layers.Activation('relu'))
six_layers_model.add(layers.Conv2D(filters=32, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_32F_2"))  # adding Convolution layer
six_layers_model.add(layers.BatchNormalization())  # adding normalization
six_layers_model.add(layers.Activation('relu'))
six_layers_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Third Part
six_layers_model.add(layers.Conv2D(filters=64, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_64F_1"))  # adding Convolution layer
six_layers_model.add(layers.BatchNormalization())  # adding normalization
six_layers_model.add(layers.Activation('relu'))
six_layers_model.add(layers.Conv2D(filters=64, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_64F_2"))  # adding Convolution layer
six_layers_model.add(layers.BatchNormalization())  # adding normalization
six_layers_model.add(layers.Activation('relu'))
six_layers_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer


# Adding dropouts
six_layers_model.add(layers.Dropout(0.30))# adding dropouts

# Flattening the output
six_layers_model.add(layers.Flatten(name="Flatten_Layer"))

# Fully Connected Layer
six_layers_model.add(layers.Dense(units=128, name="FC_Layer_128N"))
six_layers_model.add(layers.BatchNormalization())  # adding normalization
six_layers_model.add(layers.Activation('relu'))

# Adding dropouts
six_layers_model.add(layers.Dropout(0.40))  # adding dropout

# Output Layer
six_layers_model.add(layers.Dense(units=len(class_names), activation="softmax", name="Output_Layer"))  # adding output layer

# Compile the model
six_layers_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Set callbacks
learn_control = ReduceLROnPlateau(monitor='val_accuracy',
                                  patience=3,
                                  verbose=1,
                                  factor=0.1,
                                  min_lr=1e-7)

# View the summary of all layers
six_layers_model.summary()

Train the model

In [ ]:
epochs = 30
six_layers_history = six_layers_model.fit(train_ds ,
                    batch_size=batch_size ,
                    epochs=epochs,
                    validation_data=val_ds,
                    callbacks=[learn_control],
                    verbose=1)

Visualize the model results

In [ ]:
acc = six_layers_history.history['accuracy']
val_acc = six_layers_history.history['val_accuracy']

loss = six_layers_history.history['loss']
val_loss = six_layers_history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Model 8 - Eight Convolution layers with Batch Normalization and dropouts

Create the Model

In [ ]:
# Create the Model
eight_layers_model = Sequential(name="Melanoma_Detection_Model_More_Layers")

# Adding Rescaling layer
eight_layers_model.add(layers.Rescaling(1.0/255.0, offset=0.0, input_shape=(img_height, img_width, 3), name="Rescaling_Layer"))

# Adding Data Augmentation
eight_layers_model.add(data_augmentation)

# First Part
eight_layers_model.add(layers.Conv2D(filters=16, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_16F_1"))  # adding Convolution layer
eight_layers_model.add(layers.BatchNormalization())  # adding normalization
eight_layers_model.add(layers.Activation('relu'))
eight_layers_model.add(layers.Conv2D(filters=16, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_16F_2"))  # adding Convolution layer
eight_layers_model.add(layers.BatchNormalization())  # adding normalization
eight_layers_model.add(layers.Activation('relu'))
eight_layers_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Second Part
eight_layers_model.add(layers.Conv2D(filters=32, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_32F_1"))  # adding Convolution layer
eight_layers_model.add(layers.BatchNormalization())  # adding normalization
eight_layers_model.add(layers.Activation('relu'))
eight_layers_model.add(layers.Conv2D(filters=32, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_32F_2"))  # adding Convolution layer
eight_layers_model.add(layers.BatchNormalization())  # adding normalization
eight_layers_model.add(layers.Activation('relu'))
eight_layers_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Third Part
eight_layers_model.add(layers.Conv2D(filters=64, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_64F_1"))  # adding Convolution layer
eight_layers_model.add(layers.BatchNormalization())  # adding normalization
eight_layers_model.add(layers.Activation('relu'))
eight_layers_model.add(layers.Conv2D(filters=64, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_64F_2"))  # adding Convolution layer
eight_layers_model.add(layers.BatchNormalization())  # adding normalization
eight_layers_model.add(layers.Activation('relu'))
eight_layers_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Fourth Part
eight_layers_model.add(layers.Conv2D(filters=128, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_128F_1"))  # adding Convolution layer
eight_layers_model.add(layers.BatchNormalization())  # adding normalization
eight_layers_model.add(layers.Activation('relu'))
eight_layers_model.add(layers.Conv2D(filters=128, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_128F_2"))  # adding Convolution layer
eight_layers_model.add(layers.BatchNormalization())  # adding normalization
eight_layers_model.add(layers.Activation('relu'))
eight_layers_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Adding dropouts
eight_layers_model.add(layers.Dropout(0.30))# adding dropouts

# Flattening the output
eight_layers_model.add(layers.Flatten(name="Flatten_Layer"))

# Fully Connected Layer
eight_layers_model.add(layers.Dense(units=128, name="FC_Layer_128N"))
eight_layers_model.add(layers.BatchNormalization())  # adding normalization
eight_layers_model.add(layers.Activation('relu'))

# Adding dropouts
eight_layers_model.add(layers.Dropout(0.40))  # adding dropout

# Output Layer
eight_layers_model.add(layers.Dense(units=len(class_names), activation="softmax", name="Output_Layer"))  # adding output layer

# Compile the model
eight_layers_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Set callbacks
learn_control = ReduceLROnPlateau(monitor='val_accuracy',
                                  patience=3,
                                  verbose=1,
                                  factor=0.1,
                                  min_lr=1e-7)

# View the summary of all layers
eight_layers_model.summary()

Train the model

In [ ]:
epochs = 30
eight_layers_history = eight_layers_model.fit(train_ds ,
                    batch_size=batch_size ,
                    epochs=epochs,
                    validation_data=val_ds,
                    callbacks=[learn_control],
                    verbose=1)

Visualize the model results

In [ ]:
acc = eight_layers_history.history['accuracy']
val_acc = eight_layers_history.history['val_accuracy']

loss = eight_layers_history.history['loss']
val_loss = eight_layers_history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Model 9 - Ten Convolution layers with Batch Normalization and dropouts

Create the Model

In [ ]:
# Create the Model
ten_layers_model = Sequential(name="Melanoma_Detection_Model_More_Layers")

# Adding Rescaling layer
ten_layers_model.add(layers.Rescaling(1.0/255.0, offset=0.0, input_shape=(img_height, img_width, 3), name="Rescaling_Layer"))

# Adding Data Augmentation
ten_layers_model.add(data_augmentation)

# First Part
ten_layers_model.add(layers.Conv2D(filters=16, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_16F_1"))  # adding Convolution layer
ten_layers_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_model.add(layers.Activation('relu'))
ten_layers_model.add(layers.Conv2D(filters=16, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_16F_2"))  # adding Convolution layer
ten_layers_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_model.add(layers.Activation('relu'))
ten_layers_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Second Part
ten_layers_model.add(layers.Conv2D(filters=32, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_32F_1"))  # adding Convolution layer
ten_layers_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_model.add(layers.Activation('relu'))
ten_layers_model.add(layers.Conv2D(filters=32, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_32F_2"))  # adding Convolution layer
ten_layers_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_model.add(layers.Activation('relu'))
ten_layers_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Third Part
ten_layers_model.add(layers.Conv2D(filters=64, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_64F_1"))  # adding Convolution layer
ten_layers_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_model.add(layers.Activation('relu'))
ten_layers_model.add(layers.Conv2D(filters=64, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_64F_2"))  # adding Convolution layer
ten_layers_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_model.add(layers.Activation('relu'))
ten_layers_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Fourth Part
ten_layers_model.add(layers.Conv2D(filters=128, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_128F_1"))  # adding Convolution layer
ten_layers_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_model.add(layers.Activation('relu'))
ten_layers_model.add(layers.Conv2D(filters=128, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_128F_2"))  # adding Convolution layer
ten_layers_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_model.add(layers.Activation('relu'))
ten_layers_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Fifth Part
ten_layers_model.add(layers.Conv2D(filters=256, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_256F_1"))  # adding Convolution layer
ten_layers_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_model.add(layers.Activation('relu'))
ten_layers_model.add(layers.Conv2D(filters=256, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_256F_2"))  # adding Convolution layer
ten_layers_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_model.add(layers.Activation('relu'))
ten_layers_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Adding dropouts
ten_layers_model.add(layers.Dropout(0.30))# adding dropouts

# Flattening the output
ten_layers_model.add(layers.Flatten(name="Flatten_Layer"))

# Fully Connected Layer
ten_layers_model.add(layers.Dense(units=128, name="FC_Layer_128N"))
ten_layers_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_model.add(layers.Activation('relu'))

# Adding dropouts
ten_layers_model.add(layers.Dropout(0.40))  # adding dropout

# Output Layer
ten_layers_model.add(layers.Dense(units=len(class_names), activation="softmax", name="Output_Layer"))  # adding output layer

# Compile the model
ten_layers_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Set callbacks
learn_control = ReduceLROnPlateau(monitor='val_accuracy',
                                  patience=3,
                                  verbose=1,
                                  factor=0.1,
                                  min_lr=1e-7)

# View the summary of all layers
ten_layers_model.summary()

Train the model

In [ ]:
epochs = 30
ten_layers_history = ten_layers_model.fit(train_ds ,
                    batch_size=batch_size ,
                    epochs=epochs,
                    validation_data=val_ds,
                    callbacks=[learn_control],
                    verbose=1)

Visualize the model results

In [ ]:
acc = ten_layers_history.history['accuracy']
val_acc = ten_layers_history.history['val_accuracy']

loss = ten_layers_history.history['loss']
val_loss = ten_layers_history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Model 10 - Ten Convolution layers with Batch Normalization , dropouts and 50 epochs

Create the Model

In [ ]:
# Create the Model
ten_layers_50_epochs_model = Sequential(name="Melanoma_Detection_Model_More_Layers")

# Adding Rescaling layer
ten_layers_50_epochs_model.add(layers.Rescaling(1.0/255.0, offset=0.0, input_shape=(img_height, img_width, 3), name="Rescaling_Layer"))

# Adding Data Augmentation
ten_layers_50_epochs_model.add(data_augmentation)

# First Part
ten_layers_50_epochs_model.add(layers.Conv2D(filters=16, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_16F_1"))  # adding Convolution layer
ten_layers_50_epochs_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_50_epochs_model.add(layers.Activation('relu'))
ten_layers_50_epochs_model.add(layers.Conv2D(filters=16, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_16F_2"))  # adding Convolution layer
ten_layers_50_epochs_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_50_epochs_model.add(layers.Activation('relu'))
ten_layers_50_epochs_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Second Part
ten_layers_50_epochs_model.add(layers.Conv2D(filters=32, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_32F_1"))  # adding Convolution layer
ten_layers_50_epochs_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_50_epochs_model.add(layers.Activation('relu'))
ten_layers_50_epochs_model.add(layers.Conv2D(filters=32, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_32F_2"))  # adding Convolution layer
ten_layers_50_epochs_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_50_epochs_model.add(layers.Activation('relu'))
ten_layers_50_epochs_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Third Part
ten_layers_50_epochs_model.add(layers.Conv2D(filters=64, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_64F_1"))  # adding Convolution layer
ten_layers_50_epochs_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_50_epochs_model.add(layers.Activation('relu'))
ten_layers_50_epochs_model.add(layers.Conv2D(filters=64, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_64F_2"))  # adding Convolution layer
ten_layers_50_epochs_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_50_epochs_model.add(layers.Activation('relu'))
ten_layers_50_epochs_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Fourth Part
ten_layers_50_epochs_model.add(layers.Conv2D(filters=128, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_128F_1"))  # adding Convolution layer
ten_layers_50_epochs_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_50_epochs_model.add(layers.Activation('relu'))
ten_layers_50_epochs_model.add(layers.Conv2D(filters=128, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_128F_2"))  # adding Convolution layer
ten_layers_50_epochs_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_50_epochs_model.add(layers.Activation('relu'))
ten_layers_50_epochs_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Fifth Part
ten_layers_50_epochs_model.add(layers.Conv2D(filters=256, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_256F_1"))  # adding Convolution layer
ten_layers_50_epochs_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_50_epochs_model.add(layers.Activation('relu'))
ten_layers_50_epochs_model.add(layers.Conv2D(filters=256, kernel_size=(3,3),  padding="same", name="Conv2D_Layer_256F_2"))  # adding Convolution layer
ten_layers_50_epochs_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_50_epochs_model.add(layers.Activation('relu'))
ten_layers_50_epochs_model.add(layers.MaxPooling2D(pool_size=(2,2)))  # adding Max Pooling layer

# Adding dropouts
ten_layers_50_epochs_model.add(layers.Dropout(0.30))# adding dropouts

# Flattening the output
ten_layers_50_epochs_model.add(layers.Flatten(name="Flatten_Layer"))

# Fully Connected Layer
ten_layers_50_epochs_model.add(layers.Dense(units=128, name="FC_Layer_128N"))
ten_layers_50_epochs_model.add(layers.BatchNormalization())  # adding normalization
ten_layers_50_epochs_model.add(layers.Activation('relu'))

# Adding dropouts
ten_layers_50_epochs_model.add(layers.Dropout(0.40))  # adding dropout

# Output Layer
ten_layers_50_epochs_model.add(layers.Dense(units=len(class_names), activation="softmax", name="Output_Layer"))  # adding output layer

# Compile the model
ten_layers_50_epochs_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Set callbacks
learn_control = ReduceLROnPlateau(monitor='val_accuracy',
                                  patience=3,
                                  verbose=1,
                                  factor=0.1,
                                  min_lr=1e-7)

# View the summary of all layers
ten_layers_50_epochs_model.summary()

Train the model

In [ ]:
epochs = 50
ten_layers_50_epochs_history = ten_layers_50_epochs_model.fit(train_ds ,
                    batch_size=batch_size ,
                    epochs=epochs,
                    validation_data=val_ds,
                    callbacks=[learn_control],
                    verbose=1)

Visualize the model results

In [ ]:
acc = ten_layers_50_epochs_history.history['accuracy']
val_acc = ten_layers_50_epochs_history.history['val_accuracy']

loss = ten_layers_50_epochs_history.history['loss']
val_loss = ten_layers_50_epochs_history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Prediction and Evaluation on Test Set

Test dataset creation

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir_test,  # Path to the test data directory
    labels='inferred',  # Automatically infer labels from subdirectories
    label_mode='int',  # Labels will be integers
    class_names=None,  # Infer class names automatically
    color_mode='rgb',  # Images are in RGB
    batch_size=32,  # Batch size
    image_size=(img_height, img_width),  # Image size to resize to
    shuffle=False,  # Do not shuffle test data
    interpolation='bilinear',  # Interpolation method
    follow_links=False  # Follow symbolic links
)


Prediction on Train Test Validation Set

In [ ]:
# Function to evaluate accuracy on test data
def evaluate_accuracy(model_inp, data_inp):
    loss, accuracy = model_inp.evaluate(data_inp, verbose=0)  # Suppress verbose output
    return accuracy

# Dictionary containing models and their training histories
model_dict = {
    "Base Model": [base_model, base_model_history],
    "Base Model + Augmented Layers": [aug_layer_model, aug_layer_history],
    "Base Model + Augmented Layers + Balanced Class + Batch Normalization ": [class_blncd_mdl, class_balanced_history],
    "Base Model + Augmented Layers + Balanced Class + Batch Normalization + Controlled LR": [lr_control_model, lr_control_history],
    "Base Model + Augmented Layers + Balanced Class + Batch Normalization + Increased / Controlled LR": [increased_lr_ctrl_mdl, increased_lr_control_history],
    "Base Model + Augmented Layers + Balanced Class + Controlled LR + More Epochs": [more_epoch_model, more_epoch_history],
    "Six Convolution Layers + Augmented Layers + Balanced Class +  Dropouts + Batch Normalization + Controlled LR": [six_layers_model,six_layers_history],
    "Eight Convolution Layers + Augmented Layers + Balanced Class + Dropouts + Batch Normalization + Controlled LR": [eight_layers_model,eight_layers_history],
    "Ten Convolution Layers + Augmented Layers + Balanced Class + Dropouts + Batch Normalization + Controlled LR": [ten_layers_model,ten_layers_history],
    "Ten Convolution Layers + Augmented Layers + Balanced Class + Dropouts + 50 Epochs + Batch Normalization + Controlled LR": [ten_layers_50_epochs_model,ten_layers_50_epochs_history]
}

# Create an empty report dataframe with the specified columns
report_df = pd.DataFrame(columns=['Model', 'Max Train Accuracy', 'Max Validation Accuracy', 'Test Accuracy'])

# Loop through each model and extract the required metrics
for key, items in model_dict.items():
    # Get the model and history
    model = items[0]
    history = items[1]

    # Get max train accuracy and validation accuracy
    max_train_acc = max(history.history['accuracy'])  # Maximum training accuracy
    max_val_acc = max(history.history['val_accuracy'])  # Maximum validation accuracy

    # Get test accuracy by evaluating the model on the test dataset
    test_acc = evaluate_accuracy(model, test_ds)  # Accuracy on the test data

    # Create a dictionary with the metrics
    df_dict = {
        'Model': key,  # Model name
        'Max Train Accuracy': round(max_train_acc, 2),  # Max training accuracy
        'Max Validation Accuracy': round(max_val_acc, 2),  # Max validation accuracy
        'Test Accuracy': round(test_acc, 2)  # Test accuracy
    }

    # Append row directly using pandas loc to avoid warnings
    report_df.loc[len(report_df)] = df_dict  # Append row directly

# Increment index to start from 1
report_df.index += 1

# Set max col displayed
pd.set_option('display.max_colwidth', None)

# Display the final sorted report dataframe
display(report_df.sort_values(by=['Test Accuracy'], ascending=False))

## Check and note library versions

In [ ]:
print('numpy' , np.__version__)
print('pandas' ,pd.__version__)
print('matplotlib', matplotlib.__version__)
print('tensorflow' ,tf.__version__)
print('keras', keras.__version__)
print('augmentor', Augmentor.__version__)

## Conclusion

Based on the comparison of models, the following conclusions can be made:

- Model 9 (Ten Convolution Layers with Augmented Layers, Balanced Class, Dropouts, Batch Normalization, Controlled LR) achieved the highest overall performance, with a validation accuracy of 0.73 and a test accuracy of 0.50. The combination of more layers, dropout regularization, and controlled learning rates made this model the best at generalizing to unseen data.
- Model 10 (Ten Convolution Layers with Augmented Layers, Balanced Class, Dropouts, 50 Epochs, Batch Normalization, Controlled LR) had the highest training accuracy (0.72 validation accuracy, 0.48 test accuracy). Although training for more epochs improved training accuracy, the model didn’t generalize as well as Model 9, indicating that increasing the number of epochs alone may not significantly improve test performance.
- Models 6 and 8 both performed well with 0.73 validation accuracy and 0.49 test accuracy, suggesting that adding complexity in terms of convolution layers, balanced class handling, and dropout rates enhances performance.
- Model 7 (Six Convolution Layers) also showed solid results with a 0.72 validation accuracy and 0.47 test accuracy, but slightly lower than the deeper models.
- Models 4 and 5 had moderate performance, achieving validation accuracies of 0.70 and 0.65, respectively, with a test accuracy of 0.45. These models underline the importance of deeper architectures and more sophisticated regularization techniques for achieving better results.
- Base Models (1, 2, 3) suffered from overfitting, especially Model 1, which had a high training accuracy (0.91) but very low test accuracy (0.33). This confirms that deeper architectures with data augmentation and dropout regularization are essential for better generalization.

- To summarize, deeper convolutional models with controlled learning rates, batch normalization, and dropout regularization, like Model 9, performed the best. While more epochs can help in training accuracy (Model 10), they do not always lead to better test performance.

- Overall, applying data augmentation, class balancing, learning rate control, dropout optimization, and deeper architectures significantly enhanced the performance of the CNN models for melanoma detection, with Model 8 emerging as the most balanced in terms of validation and test performance. Further optimization could continue to improve this performance.

### Delete Augmented Images - /output folders

In [ ]:
import shutil  #  import shutil
for i in class_names:
    output_folder = path_to_training_dataset + i + '/output'

    # Check if the folder exists before attempting to delete
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)  # Recursively delete the entire output folder
        print(f"Deleted augmented images for class '{i}' from {output_folder}")
    else:
        print(f"No augmented images found for class '{i}', skipping deletion.")

<div align="center">------ End of Notebook -----